In [1]:
import numpy as np
import pickle
from google.colab import drive


drive.mount('/content/drive')

def save_video_dataset(X_train, y_train, X_val, y_val, train_df, val_df, base_path='/content/drive/MyDrive/accident_detection/'):
    """
    Save video frames dataset with metadata
    """
    import os
    os.makedirs(base_path, exist_ok=True)

    np.save(f'{base_path}X_train.npy', X_train)
    np.save(f'{base_path}y_train.npy', y_train)
    np.save(f'{base_path}X_val.npy', X_val)
    np.save(f'{base_path}y_val.npy', y_val)


    train_df.to_pickle(f'{base_path}train_metadata.pkl')
    val_df.to_pickle(f'{base_path}val_metadata.pkl')


    dataset_info = {
        'X_train_shape': X_train.shape,
        'X_val_shape': X_val.shape,
        'y_train_distribution': np.unique(y_train, return_counts=True),
        'y_val_distribution': np.unique(y_val, return_counts=True),
        'timestamp': pd.Timestamp.now()
    }

    with open(f'{base_path}dataset_info.pkl', 'wb') as f:
        pickle.dump(dataset_info, f)

    print(" Video dataset saved successfully!")
    print(f"X_train: {X_train.shape}")
    print(f"X_val: {X_val.shape}")
    print(f"Train metadata: {len(train_df)} videos")
    print(f"Val metadata: {len(val_df)} videos")

def load_video_dataset(base_path='/content/drive/MyDrive/accident_detection/'):
    """
    Load video frames dataset with metadata
    """

    X_train = np.load(f'{base_path}X_train.npy')
    y_train = np.load(f'{base_path}y_train.npy')
    X_val = np.load(f'{base_path}X_val.npy')
    y_val = np.load(f'{base_path}y_val.npy')


    ''' with open(f'{base_path}dataset_info.pkl', 'rb') as f:
        dataset_info = pickle.load(f)
    '''
    print("✅ Video dataset loaded successfully!")
    print(f"X_train: {X_train.shape}")
    print(f"X_val: {X_val.shape}")
    #print(f"Train metadata: {len(train_df)} videos")
    #print(f"Val metadata: {len(val_df)} videos")

    return X_train, y_train, X_val, y_val

# Save your complete dataset
#save_video_dataset(X_train, y_train, X_val, y_val, train_df, val_df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
X_train, y_train, X_val, y_val= load_video_dataset('/content/drive/MyDrive/accident_detection/')

✅ Video dataset loaded successfully!
X_train: (300, 16, 112, 112, 3)
X_val: (80, 16, 112, 112, 3)


model selectian

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np

def create_cnn_lstm_model(sequence_length=16, frame_height=112, frame_width=112, channels=3, num_classes=1):
    """
    Create a CNN-LSTM model for accident detection using pretrained DenseNet201
    """

    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))


    cnn_model = DenseNet201(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=(frame_height, frame_width, channels)
    )


    for layer in cnn_model.layers[:-30]:
        layer.trainable = False
    for layer in cnn_model.layers[-30:]:
        layer.trainable = True


    cnn_features = TimeDistributed(cnn_model)(input_layer)


    cnn_features = BatchNormalization()(cnn_features)


    lstm1 = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(lstm1)
    lstm2 = BatchNormalization()(lstm2)


    lstm3 = LSTM(32, dropout=0.1, recurrent_dropout=0.1)(lstm2)


    dense1 = Dense(64, activation='relu')(lstm3)
    dense1 = Dropout(0.3)(dense1)
    dense1 = BatchNormalization()(dense1)


    if num_classes == 1:
        output_layer = Dense(1, activation='sigmoid')(dense1)
    else:
        output_layer = Dense(num_classes, activation='softmax')(dense1)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model


model = create_cnn_lstm_model(sequence_length=16, frame_height=112, frame_width=112, channels=3, num_classes=1)


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)


model.summary()


callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    ModelCheckpoint('best_accident_model.h5', save_best_only=True, monitor='val_accuracy')
]


print("Starting training...")
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    shuffle=True,
    verbose=1
)

print("Training completed!")

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 16, 112, 112,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 16, 1920)       │    18,321,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 1920)       │         7,680 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 16, 128)        │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 16, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,443,777 (74.17 MB)

 Trainable params: 2,222,977 (8.48 MB)

 Non-trainable params: 17,220,800 (65.69 MB)

Starting training...
Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - accuracy: 0.4528 - loss: 0.9303 - precision: 0.4958 - recall: 0.4875

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from google.colab import drive
import os
import numpy as np


drive.mount('/content/drive')


drive_model_path = '/content/drive/MyDrive/accident_detection_models/'
os.makedirs(drive_model_path, exist_ok=True)

def create_cnn_lstm_model(sequence_length=16, frame_height=112, frame_width=112, channels=3, num_classes=1):
    """
    Create a CNN-LSTM model for accident detection using pretrained DenseNet201
    """

    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))


    cnn_model = DenseNet201(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=(frame_height, frame_width, channels)
    )


    for layer in cnn_model.layers[:-30]:
        layer.trainable = False
    for layer in cnn_model.layers[-30:]:
        layer.trainable = True


    cnn_features = TimeDistributed(cnn_model)(input_layer)


    cnn_features = BatchNormalization()(cnn_features)


    lstm1 = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(lstm1)
    lstm2 = BatchNormalization()(lstm2)


    lstm3 = LSTM(32, dropout=0.1, recurrent_dropout=0.1)(lstm2)


    dense1 = Dense(64, activation='relu')(lstm3)
    dense1 = Dropout(0.3)(dense1)
    dense1 = BatchNormalization()(dense1)


    if num_classes == 1:
        output_layer = Dense(1, activation='sigmoid')(dense1)
    else:
        output_layer = Dense(num_classes, activation='softmax')(dense1)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model


model = create_cnn_lstm_model(sequence_length=16, frame_height=112, frame_width=112, channels=3, num_classes=1)


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)


model.summary()


callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),

    ModelCheckpoint(
        drive_model_path + 'best_accident_model.h5',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )
]

print(f"Models will be saved to: {drive_model_path}")


print("Starting training...")
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    shuffle=True,
    verbose=1
)

print("Training completed!")


final_model_path = drive_model_path + 'final_accident_model.h5'
model.save(final_model_path)
print(f"Final model saved to: {final_model_path}")


history_path = drive_model_path + 'training_history.npy'
np.save(history_path, history.history)
print(f"Training history saved to: {history_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 112, 112,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 16, 1920)       │    18,321,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 1920)       │         7,680 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16, 128)        │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,443,777 (74.17 MB)

 Trainable params: 2,222,977 (8.48 MB)

...

38/38 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.5603 - loss: 0.7490 - precision: 0.6198 - recall: 0.5410 - val_accuracy: 0.6125 - val_loss: 0.6744 - val_precision: 0.6486 - val_recall: 0.5714 - learning_rate: 1.0000e-04
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.6767 - loss: 0.7284 - precision: 0.6725 - recall: 0.6924
Epoch 4: val_accuracy improved from 0.61250 to 0.65000, saving model to /content/drive/MyDrive/accident_detection_models/best_accident_model.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.6757 - loss: 0.7281 - precision: 0.6716 - recall: 0.6910 - val_accuracy: 0.6500 - val_loss: 0.6587 - val_precision: 0.7188 - val_recall: 0.5476 - learning_rate: 1.0000e-04
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 779ms/step - accuracy: 0.6006 - loss: 0.7146 - precision: 0.6185 - recall: 0.5675
Epoch 5: val_accuracy improved from 0.65000 to 0.67500, saving model to /content/drive/MyDrive/accident_detection_models/best_accident_model.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6014 - loss: 0.7138 - precision: 0.6191 - recall: 0.5688 - val_accuracy: 0.6750 - val_loss: 0.6483 - val_precision: 0.7105 - val_recall: 0.6429 - learning_rate: 1.0000e-04
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 724ms/step - accuracy: 0.6630 - loss: 0.6734 - precision: 0.6905 - recall: 0.6331
Epoch 6: val_accuracy did not improve from 0.67500
38/38 ━━━━━━━━━━━━━━━━━━━━ 74s 866ms/step - accuracy: 0.6624 - loss: 0.6735 - precision: 0.6893 - recall: 0.6332 - val_accuracy: 0.6750 - val_loss: 0.6616 - val_precision: 0.7667 - val_recall: 0.5476 - learning_rate: 1.0000e-04
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 743ms/step - accuracy: 0.6116 - loss: 0.6743 - precision: 0.5764 - recall: 0.6740
Epoch 7: val_accuracy did not improve from 0.67500
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 894ms/step - accuracy: 0.6121 - loss: 0.6746 - precision: 0.5776 - recall: 0.6741 - val_accuracy: 0.6625 - val_loss: 0.6351 - val_precision: 0.7273 - val_recall: 0.5714

38/38 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.6888 - loss: 0.6216 - precision: 0.6747 - recall: 0.7063 - val_accuracy: 0.7125 - val_loss: 0.6079 - val_precision: 0.7714 - val_recall: 0.6429 - learning_rate: 1.0000e-04
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.6641 - loss: 0.6375 - precision: 0.6893 - recall: 0.6649
Epoch 9: val_accuracy did not improve from 0.71250
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 889ms/step - accuracy: 0.6643 - loss: 0.6369 - precision: 0.6889 - recall: 0.6650 - val_accuracy: 0.6375 - val_loss: 0.6656 - val_precision: 0.7097 - val_recall: 0.5238 - learning_rate: 1.0000e-04
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 748ms/step - accuracy: 0.7436 - loss: 0.5617 - precision: 0.7279 - recall: 0.7539
Epoch 10: val_accuracy did not improve from 0.71250
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 904ms/step - accuracy: 0.7435 - loss: 0.5616 - precision: 0.7284 - recall: 0.7535 - val_accuracy: 0.5750 - val_loss: 0.7001 - val_precision: 0.6538 - val_recall: 0.40

Training completed!
Final model saved to: /content/drive/MyDrive/accident_detection_models/final_accident_model.h5
Training history saved to: /content/drive/MyDrive/accident_detection_models/training_history.npy


In [8]:
def save_model_to_drive(model, model_name="accident_model"):
    """Manually save model to Google Drive"""
    drive_model_path = f'/content/drive/MyDrive/accident_detection_models/{model_name}.h5'
    model.save(drive_model_path)
    print(f" Model saved to: {drive_model_path}")
    return drive_model_path

def load_model_from_drive(model_name="accident_model"):
    """Load model from Google Drive"""
    from tensorflow.keras.models import load_model
    drive_model_path = f'/content/drive/MyDrive/accident_detection_models/{model_name}.h5'
    model = load_model(drive_model_path)
    print(f" Model loaded from: {drive_model_path}")
    return model


save_model_to_drive(model, "best_accident_model")



NameError: name 'model' is not defined

In [10]:
from google.colab import drive
from tensorflow.keras.models import load_model

# Mount Drive
drive.mount('/content/drive')



loaded_model = load_model_from_drive("best_accident_model1")
print("✅ Model loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


✅ Model loaded from: /content/drive/MyDrive/accident_detection_models/best_accident_model1.h5
✅ Model loaded successfully!


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization, GlobalAveragePooling2D, ConvLSTM2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import AdamW
import numpy as np

def create_improved_accident_model(sequence_length=16, frame_height=112, frame_width=112, channels=3):
    """
    Improved CNN-LSTM model with better architecture for accident detection
    """
    # Input layer
    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))

    # Use EfficientNetB0 - lighter and more efficient than DenseNet201
    cnn_base = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(frame_height, frame_width, channels)
    )

    # Freeze more layers initially (we'll unfreeze later if needed)
    for layer in cnn_base.layers[:-50]:
        layer.trainable = False

    # Create feature extractor with Global Average Pooling
    cnn_features = TimeDistributed(cnn_base)(input_layer)
    cnn_features = TimeDistributed(GlobalAveragePooling2D())(cnn_features)

    # Better normalization and dropout
    cnn_features = BatchNormalization()(cnn_features)
    cnn_features = Dropout(0.3)(cnn_features)

    # Improved LSTM architecture with bidirectional layers
    lstm1 = LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3,
                 kernel_regularizer=l2(0.01))(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3,
                 kernel_regularizer=l2(0.01))(lstm1)
    lstm2 = BatchNormalization()(lstm2)

    lstm3 = LSTM(64, dropout=0.2, recurrent_dropout=0.2,
                 kernel_regularizer=l2(0.01))(lstm2)

    # Enhanced classification head
    dense1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(lstm3)
    dense1 = BatchNormalization()(dense1)
    dense1 = Dropout(0.4)(dense1)

    dense2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Dropout(0.3)(dense2)

    # Output layer
    output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create the improved model
model = create_improved_accident_model(sequence_length=16, frame_height=112, frame_width=112, channels=3)

# Improved compilation with better optimizer and metrics
model.compile(
    optimizer=AdamW(learning_rate=0.001, weight_decay=0.004),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_video_augmentation():
    """Create data augmentation for video sequences"""
    return tf.keras.Sequential([
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.2),
        # Note: We don't flip horizontally for traffic scenes (would change direction)
    ])

# Apply augmentation to training data
augmentation_model = create_video_augmentation()

def augment_video_sequence(X_batch):
    """Apply augmentation to a batch of video sequences"""
    batch_size, seq_len, height, width, channels = X_batch.shape
    X_batch = tf.reshape(X_batch, [-1, height, width, channels])
    X_batch = augmentation_model(X_batch, training=True)
    X_batch = tf.reshape(X_batch, [batch_size, seq_len, height, width, channels])
    return X_batch

In [ ]:
class CustomTrainingSchedule:
    """Custom training schedule with progressive unfreezing"""
    def __init__(self, model):
        self.model = model
        self.unfreeze_layers = False

    def on_epoch_end(self, epoch):
        if epoch == 10 and not self.unfreeze_layers:
            # Unfreeze more CNN layers after 10 epochs
            for layer in self.model.layers[1].layers:  # TimeDistributed layer
                if hasattr(layer, 'layers'):
                    for cnn_layer in layer.layers[-20:]:
                        cnn_layer.trainable = True
            self.unfreeze_layers = True
            print("Unfrozen last 20 CNN layers for fine-tuning")

# Enhanced callbacks
callbacks = [
    EarlyStopping(
        patience=20,
        restore_best_weights=True,
        monitor='val_auc',  # Monitor AUC for better overall performance
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        '/content/drive/MyDrive/accident_detection_models/best_improved_model.h5',
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
]

# Handle class imbalance (if any)
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print(f"Class weights: {class_weight_dict}")

# Custom training loop with augmentation
def train_with_augmentation(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=8):
    """Train with custom augmentation"""
    custom_schedule = CustomTrainingSchedule(model)

    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        class_weight=class_weight_dict,
        shuffle=True,
        verbose=1
    )
    return history

print("Starting improved training...")
history = train_with_augmentation(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=8)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np

def create_resnet_lstm_model(sequence_length=16, frame_height=112, frame_width=112, channels=3):
    """
    ResNet50 + LSTM model for accident detection
    """
    # Input layer for video sequences
    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))

    # ResNet50 - Excellent balance of performance and efficiency
    resnet_base = ResNet50(
        weights='imagenet',
        include_top=False,
        pooling='avg',  # Global average pooling
        input_shape=(frame_height, frame_width, channels)
    )

    # Freeze early layers, fine-tune later layers
    # ResNet50 has 175 layers - we'll freeze the first 100
    for layer in resnet_base.layers[:100]:
        layer.trainable = False
    for layer in resnet_base.layers[100:]:
        layer.trainable = True

    print(f"ResNet50 layers: {len(resnet_base.layers)}")
    print(f"Trainable layers: {sum([layer.trainable for layer in resnet_base.layers])}")

    # Apply ResNet to each frame
    cnn_features = TimeDistributed(resnet_base)(input_layer)

    # Batch normalization after feature extraction
    cnn_features = BatchNormalization()(cnn_features)

    # LSTM layers for temporal analysis
    lstm1 = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(lstm1)
    lstm2 = BatchNormalization()(lstm2)

    lstm3 = LSTM(32, dropout=0.1, recurrent_dropout=0.1)(lstm2)

    # Classification head
    dense1 = Dense(64, activation='relu')(lstm3)
    dense1 = Dropout(0.3)(dense1)

    output_layer = Dense(1, activation='sigmoid')(dense1)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create ResNet model
model = create_resnet_lstm_model()

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall', 'auc']
)

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
ResNet50 layers: 176
Trainable layers: 76


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 112, 112,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 16, 2048)       │    23,587,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 2048)       │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16, 128)        │     1,114,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,775,297 (94.51 MB)

 Trainable params: 20,636,033 (78.72 MB)

 Non-trainable params: 4,139,264 (15.79 MB)

In [12]:
# Improved callbacks for ResNet
callbacks = [
    EarlyStopping(
        patience=15,
        restore_best_weights=True,
        monitor='val_auc',  # Monitor AUC for better performance measurement
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        '/content/drive/MyDrive/accident_detection_models/best_resnet_model.h5',
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
]

# Training with ResNet
print("Training ResNet50 + LSTM model...")
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    shuffle=True,
    verbose=1
)

Training ResNet50 + LSTM model...
Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 711ms/step - accuracy: 0.5263 - auc: 0.5361 - loss: 0.7030 - precision: 0.5628 - recall: 0.4666
Epoch 1: val_auc improved from -inf to 0.53008, saving model to /content/drive/MyDrive/accident_detection_models/best_resnet_model.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.5262 - auc: 0.5357 - loss: 0.7031 - precision: 0.5619 - recall: 0.4669 - val_accuracy: 0.5250 - val_auc: 0.5301 - val_loss: 0.6926 - val_precision: 0.5270 - val_recall: 0.9286 - learning_rate: 1.0000e-04
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 719ms/step - accuracy: 0.5860 - auc: 0.6236 - loss: 0.6729 - precision: 0.5845 - recall: 0.5857
Epoch 2: val_auc improved from 0.53008 to 0.54167, saving model to /content/drive/MyDrive/accident_detection_models/best_resnet_model.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 36s 946ms/step - accuracy: 0.5861 - auc: 0.6235 - loss: 0.6729 - precision: 0.5847 - recall: 0.5860 - val_accuracy: 0.5000 - val_auc: 0.5417 - val_loss: 0.6902 - val_precision: 0.5385 - val_recall: 0.3333 - learning_rate: 1.0000e-04
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 809ms/step - accuracy: 0.5812 - auc: 0.6192 - loss: 0.6666 - precision: 0.6264 - recall: 0.5573
Epoch 3: val_auc improved from 0.54167 to 0.63283, saving model to /content/drive/MyDrive/accident_detection_models/best_resnet_model.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 36s 960ms/step - accuracy: 0.5805 - auc: 0.6186 - loss: 0.6668 - precision: 0.6248 - recall: 0.5568 - val_accuracy: 0.6250 - val_auc: 0.6328 - val_loss: 0.6836 - val_precision: 0.6429 - val_recall: 0.6429 - learning_rate: 1.0000e-04
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 758ms/step - accuracy: 0.5605 - auc: 0.6182 - loss: 0.6679 - precision: 0.5247 - recall: 0.4627
Epoch 4: val_auc did not improve from 0.63283
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 835ms/step - accuracy: 0.5604 - auc: 0.6181 - loss: 0.6679 - precision: 0.5260 - recall: 0.4627 - val_accuracy: 0.5750 - val_auc: 0.6244 - val_loss: 0.6798 - val_precision: 0.6111 - val_recall: 0.5238 - learning_rate: 1.0000e-04
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - accuracy: 0.5684 - auc: 0.5925 - loss: 0.6770 - precision: 0.5174 - recall: 0.5594
Epoch 5: val_auc did not improve from 0.63283
38/38 ━━━━━━━━━━━━━━━━━━━━ 31s 804ms/step - accuracy: 0.5681 - auc: 0.5921 - loss: 0.6772 - precision: 0.5185 - re

In [1]:
def create_densenet_progressive(sequence_length=16, frame_height=112, frame_width=112, channels=3):
    """
    DenseNet with progressive unfreezing for better feature learning
    """
    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))

    # DenseNet201 - start completely frozen
    densenet_base = DenseNet201(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=(frame_height, frame_width, channels)
    )

    # Initially freeze all layers
    for layer in densenet_base.layers:
        layer.trainable = False

    cnn_features = TimeDistributed(densenet_base)(input_layer)
    cnn_features = BatchNormalization()(cnn_features)

    # LSTM layers
    lstm1 = LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(lstm1)
    lstm2 = BatchNormalization()(lstm2)

    lstm3 = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(lstm2)

    # Enhanced classification head
    dense1 = Dense(128, activation='relu')(lstm3)
    dense1 = BatchNormalization()(dense1)
    dense1 = Dropout(0.4)(dense1)

    dense2 = Dense(64, activation='relu')(dense1)
    dense2 = Dropout(0.3)(dense2)

    output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Two-phase training approach
def train_progressive(model, X_train, y_train, X_val, y_val):
    """
    Phase 1: Train with frozen backbone
    Phase 2: Unfreeze and fine-tune last layers
    """
    # Phase 1: Frozen backbone
    print("=== PHASE 1: Training with frozen backbone ===")
    history1 = model.fit(
        X_train, y_train,
        batch_size=8,
        epochs=15,
        validation_data=(X_val, y_val),
        callbacks=[
            EarlyStopping(patience=8, restore_best_weights=True, monitor='val_accuracy'),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)
        ],
        verbose=1
    )

    # Phase 2: Unfreeze last 50 layers of DenseNet
    print("=== PHASE 2: Fine-tuning last layers ===")
    densenet_layers = model.layers[1].layer.layers  # TimeDistributed -> DenseNet layers
    for layer in densenet_layers[-50:]:
        layer.trainable = True

    # Recompile with lower learning rate for fine-tuning
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.00001),  # 10x lower LR
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    history2 = model.fit(
        X_train, y_train,
        batch_size=8,
        epochs=30,
        validation_data=(X_val, y_val),
        callbacks=[
            EarlyStopping(patience=10, restore_best_weights=True, monitor='val_accuracy'),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
        ],
        verbose=1
    )

    return history1, history2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
import numpy as np
import matplotlib.pyplot as plt

# Mount Google Drive and load your data
from google.colab import drive
drive.mount('/content/drive')

# Load your dataset
def load_video_dataset(base_path='/content/drive/MyDrive/accident_detection/'):
    X_train = np.load(f'{base_path}X_train.npy')
    y_train = np.load(f'{base_path}y_train.npy')
    X_val = np.load(f'{base_path}X_val.npy')
    y_val = np.load(f'{base_path}y_val.npy')

    print(" Dataset loaded successfully!")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    return X_train, y_train, X_val, y_val

# Load your data
X_train, y_train, X_val, y_val = load_video_dataset()

def create_densenet_progressive(sequence_length=16, frame_height=112, frame_width=112, channels=3):
    """
    DenseNet with progressive unfreezing for better feature learning
    """
    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))

    # DenseNet201 - start completely frozen
    densenet_base = DenseNet201(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=(frame_height, frame_width, channels)
    )

    # Initially freeze all layers
    for layer in densenet_base.layers:
        layer.trainable = False

    print(f" DenseNet201 loaded with {len(densenet_base.layers)} layers")
    print(f" Initially frozen layers: {sum([not layer.trainable for layer in densenet_base.layers])}")

    cnn_features = TimeDistributed(densenet_base)(input_layer)
    cnn_features = BatchNormalization()(cnn_features)
    cnn_features = Dropout(0.2)(cnn_features)

    # Enhanced LSTM layers
    lstm1 = LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)(lstm1)
    lstm2 = BatchNormalization()(lstm2)

    lstm3 = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(lstm2)

    # Enhanced classification head
    dense1 = Dense(128, activation='relu')(lstm3)
    dense1 = BatchNormalization()(dense1)
    dense1 = Dropout(0.4)(dense1)

    dense2 = Dense(64, activation='relu')(dense1)
    dense2 = Dropout(0.3)(dense2)

    output_layer = Dense(1, activation='sigmoid')(dense2)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

def train_progressive(model, X_train, y_train, X_val, y_val):
    """
    Two-phase training approach
    """
    # Callbacks for both phases
    phase1_callbacks = [
        EarlyStopping(patience=8, restore_best_weights=True, monitor='val_accuracy', verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1),
        ModelCheckpoint('/content/drive/MyDrive/accident_detection_models/phase1_best.h5',
                       save_best_only=True, monitor='val_accuracy', verbose=1)
    ]

    phase2_callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True, monitor='val_accuracy', verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
        ModelCheckpoint('/content/drive/MyDrive/accident_detection_models/phase2_best.h5',
                       save_best_only=True, monitor='val_accuracy', verbose=1)
    ]

    # Phase 1: Frozen backbone
    print("=" * 60)
    print("PHASE 1: Training with frozen DenseNet backbone")
    print("=" * 60)

    # Compile for Phase 1
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )

    history1 = model.fit(
        X_train, y_train,
        batch_size=8,
        epochs=20,  # Increased epochs for Phase 1
        validation_data=(X_val, y_val),
        callbacks=phase1_callbacks,
        verbose=1
    )

    # Phase 2: Unfreeze last 80 layers of DenseNet for fine-tuning
    print("\n" + "=" * 60)
    print("PHASE 2: Fine-tuning last 80 DenseNet layers")
    print("=" * 60)

    # Get the DenseNet model inside TimeDistributed
    time_distributed_layer = model.layers[1]
    densenet_layers = time_distributed_layer.layer.layers

    # Unfreeze last 80 layers
    layers_unfrozen = 0
    for i, layer in enumerate(densenet_layers[-80:], 1):
        layer.trainable = True
        layers_unfrozen += 1

    print(f" Unfrozen {layers_unfrozen} layers for fine-tuning")
    print(f" Trainable layers: {sum([layer.trainable for layer in densenet_layers])}/{len(densenet_layers)}")

    # Recompile with lower learning rate for fine-tuning
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.00001),  # 10x lower LR
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )

    history2 = model.fit(
        X_train, y_train,
        batch_size=8,
        epochs=40,  # More epochs for fine-tuning
        validation_data=(X_val, y_val),
        callbacks=phase2_callbacks,
        verbose=1
    )

    return history1, history2, model

def plot_training_results(history1, history2):
    """Plot training results from both phases"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Combine histories
    total_epochs = len(history1.history['accuracy']) + len(history2.history['accuracy'])

    # Accuracy plot
    axes[0, 0].plot(history1.history['accuracy'], label='Phase 1 Train', color='blue')
    axes[0, 0].plot(history1.history['val_accuracy'], label='Phase 1 Val', color='blue', linestyle='--')
    axes[0, 0].plot(range(len(history1.history['accuracy']), total_epochs),
                   history2.history['accuracy'], label='Phase 2 Train', color='red')
    axes[0, 0].plot(range(len(history1.history['val_accuracy']), total_epochs),
                   history2.history['val_accuracy'], label='Phase 2 Val', color='red', linestyle='--')
    axes[0, 0].axvline(x=len(history1.history['accuracy']), color='black', linestyle=':', alpha=0.7, label='Phase Switch')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Loss plot
    axes[0, 1].plot(history1.history['loss'], label='Phase 1 Train', color='blue')
    axes[0, 1].plot(history1.history['val_loss'], label='Phase 1 Val', color='blue', linestyle='--')
    axes[0, 1].plot(range(len(history1.history['loss']), total_epochs),
                   history2.history['loss'], label='Phase 2 Train', color='red')
    axes[0, 1].plot(range(len(history1.history['val_loss']), total_epochs),
                   history2.history['val_loss'], label='Phase 2 Val', color='red', linestyle='--')
    axes[0, 1].axvline(x=len(history1.history['loss']), color='black', linestyle=':', alpha=0.7, label='Phase Switch')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Precision plot
    axes[1, 0].plot(history1.history['precision'], label='Phase 1 Train', color='blue')
    axes[1, 0].plot(history1.history['val_precision'], label='Phase 1 Val', color='blue', linestyle='--')
    axes[1, 0].plot(range(len(history1.history['precision']), total_epochs),
                   history2.history['precision'], label='Phase 2 Train', color='red')
    axes[1, 0].plot(range(len(history1.history['val_precision']), total_epochs),
                   history2.history['val_precision'], label='Phase 2 Val', color='red', linestyle='--')
    axes[1, 0].axvline(x=len(history1.history['precision']), color='black', linestyle=':', alpha=0.7)
    axes[1, 0].set_title('Model Precision')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Recall plot
    axes[1, 1].plot(history1.history['recall'], label='Phase 1 Train', color='blue')
    axes[1, 1].plot(history1.history['val_recall'], label='Phase 1 Val', color='blue', linestyle='--')
    axes[1, 1].plot(range(len(history1.history['recall']), total_epochs),
                   history2.history['recall'], label='Phase 2 Train', color='red')
    axes[1, 1].plot(range(len(history1.history['val_recall']), total_epochs),
                   history2.history['val_recall'], label='Phase 2 Val', color='red', linestyle='--')
    axes[1, 1].axvline(x=len(history1.history['recall']), color='black', linestyle=':', alpha=0.7)
    axes[1, 1].set_title('Model Recall')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Main execution
print("🚀 Starting Progressive Unfreezing Strategy")
print("=" * 60)

# Create model
model = create_densenet_progressive()
model.summary()

# Train with progressive unfreezing
history1, history2, final_model = train_progressive(model, X_train, y_train, X_val, y_val)

# Plot results
plot_training_results(history1, history2)

# Final evaluation
print("\n" + "=" * 60)
print("FINAL EVALUATION")
print("=" * 60)

final_loss, final_accuracy, final_precision, final_recall, final_auc = final_model.evaluate(X_val, y_val, verbose=0)
print(f" Final Validation Accuracy: {final_accuracy:.4f}")
print(f" Final Validation Precision: {final_precision:.4f}")
print(f" Final Validation Recall: {final_recall:.4f}")
print(f" Final Validation AUC: {final_auc:.4f}")

# Save final model
final_model_path = '/content/drive/MyDrive/accident_detection_models/progressive_unfreezing_final.h5'
final_model.save(final_model_path)
print(f" Final model saved to: {final_model_path}")

print("\n Training completed! Check the plots to see if Phase 2 improved performance.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Dataset loaded successfully!
X_train: (300, 16, 112, 112, 3), y_train: (300,)
X_val: (80, 16, 112, 112, 3), y_val: (80,)
🚀 Starting Progressive Unfreezing Strategy
 DenseNet201 loaded with 708 layers
 Initially frozen layers: 708


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 112, 112,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 16, 1920)       │    18,321,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 1920)       │         7,680 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 1920)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16, 256)        │     2,229,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16, 128)        │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,824,129 (79.44 MB)

 Trainable params: 2,497,281 (9.53 MB)

...

PHASE 1: Training with frozen DenseNet backbone
Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 713ms/step - accuracy: 0.4595 - auc: 0.5891 - loss: 1.1280 - precision: 0.5002 - recall: 0.1183
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /content/drive/MyDrive/accident_detection_models/phase1_best.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 510s 5s/step - accuracy: 0.4606 - auc: 0.5891 - loss: 1.1255 - precision: 0.5005 - recall: 0.1188 - val_accuracy: 0.5000 - val_auc: 0.5119 - val_loss: 0.6946 - val_precision: 0.6000 - val_recall: 0.1429 - learning_rate: 1.0000e-04
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 702ms/step - accuracy: 0.5456 - auc: 0.5255 - loss: 1.0735 - precision: 0.6146 - recall: 0.2078
Epoch 2: val_accuracy improved from 0.50000 to 0.52500, saving model to /content/drive/MyDrive/accident_detection_models/phase1_best.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 36s 964ms/step - accuracy: 0.5444 - auc: 0.5242 - loss: 1.0753 - precision: 0.6117 - recall: 0.2067 - val_accuracy: 0.5250 - val_auc: 0.5407 - val_loss: 0.6942 - val_precision: 0.6429 - val_recall: 0.2143 - learning_rate: 1.0000e-04
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 708ms/step - accuracy: 0.4822 - auc: 0.4950 - loss: 1.1549 - precision: 0.4175 - recall: 0.1180
Epoch 3: val_accuracy did not improve from 0.52500
38/38 ━━━━━━━━━━━━━━━━━━━━ 37s 866ms/step - accuracy: 0.4831 - auc: 0.4964 - loss: 1.1516 - precision: 0.4215 - recall: 0.1198 - val_accuracy: 0.5250 - val_auc: 0.5739 - val_loss: 0.6923 - val_precision: 0.7000 - val_recall: 0.1667 - learning_rate: 1.0000e-04
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 678ms/step - accuracy: 0.4771 - auc: 0.5028 - loss: 1.0564 - precision: 0.4270 - recall: 0.1689
Epoch 4: val_accuracy did not improve from 0.52500
38/38 ━━━━━━━━━━━━━━━━━━━━ 40s 834ms/step - accuracy: 0.4785 - auc: 0.5036 - loss: 1.0559 - precision: 0

38/38 ━━━━━━━━━━━━━━━━━━━━ 35s 941ms/step - accuracy: 0.5020 - auc: 0.4852 - loss: 1.0664 - precision: 0.4056 - recall: 0.1638 - val_accuracy: 0.5500 - val_auc: 0.6920 - val_loss: 0.6779 - val_precision: 0.8000 - val_recall: 0.1905 - learning_rate: 1.0000e-04
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 695ms/step - accuracy: 0.5158 - auc: 0.4818 - loss: 1.0794 - precision: 0.5999 - recall: 0.2072
Epoch 6: val_accuracy improved from 0.55000 to 0.58750, saving model to /content/drive/MyDrive/accident_detection_models/phase1_best.h5


38/38 ━━━━━━━━━━━━━━━━━━━━ 36s 945ms/step - accuracy: 0.5163 - auc: 0.4831 - loss: 1.0766 - precision: 0.5999 - recall: 0.2080 - val_accuracy: 0.5875 - val_auc: 0.7215 - val_loss: 0.6674 - val_precision: 0.8462 - val_recall: 0.2619 - learning_rate: 1.0000e-04
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 693ms/step - accuracy: 0.5480 - auc: 0.6001 - loss: 0.9100 - precision: 0.6435 - recall: 0.2699
Epoch 7: val_accuracy did not improve from 0.58750
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 848ms/step - accuracy: 0.5482 - auc: 0.6000 - loss: 0.9098 - precision: 0.6432 - recall: 0.2702 - val_accuracy: 0.5625 - val_auc: 0.7061 - val_loss: 0.6714 - val_precision: 0.7333 - val_recall: 0.2619 - learning_rate: 1.0000e-04
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 678ms/step - accuracy: 0.5864 - auc: 0.6571 - loss: 0.7631 - precision: 0.5872 - recall: 0.2647
Epoch 8: val_accuracy did not improve from 0.58750
38/38 ━━━━━━━━━━━━━━━━━━━━ 40s 829ms/step - accuracy: 0.5859 - auc: 0.6563 - loss: 0.7649 - precision: 0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from tensorflow.keras.applications import DenseNet121  # Using smaller DenseNet
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt

# Mount Google Drive and load data
from google.colab import drive
drive.mount('/content/drive')

def load_video_dataset(base_path='/content/drive/MyDrive/accident_detection/'):
    X_train = np.load(f'{base_path}X_train.npy')
    y_train = np.load(f'{base_path}y_train.npy')
    X_val = np.load(f'{base_path}X_val.npy')
    y_val = np.load(f'{base_path}y_val.npy')

    print("✅ Dataset loaded successfully!")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    return X_train, y_train, X_val, y_val

# Load your data
X_train, y_train, X_val, y_val = load_video_dataset()

def create_memory_efficient_model(sequence_length=16, frame_height=112, frame_width=112, channels=3):
    """
    Memory-efficient version with smaller DenseNet121 and optimized architecture
    """
    input_layer = Input(shape=(sequence_length, frame_height, frame_width, channels))

    # Use DenseNet121 instead of DenseNet201 (smaller, less memory)
    densenet_base = DenseNet121(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=(frame_height, frame_width, channels)
    )

    # Initially freeze all layers
    for layer in densenet_base.layers:
        layer.trainable = False

    print(f"✅ DenseNet121 loaded with {len(densenet_base.layers)} layers")
    print(f"✅ Initially frozen layers: {sum([not layer.trainable for layer in densenet_base.layers])}")

    cnn_features = TimeDistributed(densenet_base)(input_layer)
    cnn_features = BatchNormalization()(cnn_features)
    cnn_features = Dropout(0.2)(cnn_features)

    # Smaller LSTM layers to save memory
    lstm1 = LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(cnn_features)
    lstm1 = BatchNormalization()(lstm1)

    lstm2 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(lstm1)
    lstm2 = BatchNormalization()(lstm2)

    lstm3 = LSTM(32, dropout=0.1, recurrent_dropout=0.1)(lstm2)

    # Classification head
    dense1 = Dense(64, activation='relu')(lstm3)
    dense1 = Dropout(0.3)(dense1)

    output_layer = Dense(1, activation='sigmoid')(dense1)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

def train_memory_safe(model, X_train, y_train, X_val, y_val):
    """
    Memory-safe training with smaller batches and gradual unfreezing
    """
    # Phase 1: Only train classification head
    print("=" * 60)
    print("PHASE 1: Training classification head only")
    print("=" * 60)

    # Ensure all DenseNet layers are frozen
    time_distributed_layer = model.layers[1]
    densenet_layers = time_distributed_layer.layer.layers
    for layer in densenet_layers:
        layer.trainable = False

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    phase1_callbacks = [
        EarlyStopping(patience=6, restore_best_weights=True, monitor='val_accuracy', verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
        ModelCheckpoint('/content/drive/MyDrive/accident_detection_models/phase1_safe.h5',
                       save_best_only=True, monitor='val_accuracy')
    ]

    # Use smaller batch size for memory safety
    history1 = model.fit(
        X_train, y_train,
        batch_size=4,  # Reduced batch size
        epochs=15,
        validation_data=(X_val, y_val),
        callbacks=phase1_callbacks,
        verbose=1
    )

    # Phase 2: Unfreeze only the last FEW layers (more conservative)
    print("\n" + "=" * 60)
    print("PHASE 2: Fine-tuning last 20 DenseNet layers")
    print("=" * 60)

    # Unfreeze only last 20 layers (very conservative)
    layers_unfrozen = 0
    for layer in densenet_layers[-20:]:
        layer.trainable = True
        layers_unfrozen += 1

    print(f"✅ Unfrozen {layers_unfrozen} layers for fine-tuning")

    # Recompile with very low learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.00001),  # Very low LR
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    phase2_callbacks = [
        EarlyStopping(patience=8, restore_best_weights=True, monitor='val_accuracy', verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1),
        ModelCheckpoint('/content/drive/MyDrive/accident_detection_models/phase2_safe.h5',
                       save_best_only=True, monitor='val_accuracy')
    ]

    # Use even smaller batch size for Phase 2
    history2 = model.fit(
        X_train, y_train,
        batch_size=4,  # Small batch size
        epochs=25,
        validation_data=(X_val, y_val),
        callbacks=phase2_callbacks,
        verbose=1
    )

    return history1, history2, model

# Alternative: Simple Single-Phase Training (Most Reliable)
def train_simple_effective(model, X_train, y_train, X_val, y_val):
    """
    Simple single-phase training that's guaranteed to work on T4
    """
    print("=" * 60)
    print("SIMPLE SINGLE-PHASE TRAINING (Most Reliable)")
    print("=" * 60)

    # Keep DenseNet completely frozen
    time_distributed_layer = model.layers[1]
    densenet_layers = time_distributed_layer.layer.layers
    for layer in densenet_layers:
        layer.trainable = False

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )

    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True, monitor='val_accuracy', verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1),
        ModelCheckpoint('/content/drive/MyDrive/accident_detection_models/simple_best.h5',
                       save_best_only=True, monitor='val_accuracy')
    ]

    history = model.fit(
        X_train, y_train,
        batch_size=8,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )

    return history, model

def plot_results(history):
    """Plot training results"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0, 0].plot(history.history['val_accuracy'], label='Val Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train Loss')
    axes[0, 1].plot(history.history['val_loss'], label='Val Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Precision')
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision')
    axes[1, 0].set_title('Model Precision')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall')
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall')
    axes[1, 1].set_title('Model Recall')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Main execution - CHOOSE ONE APPROACH:

print("🚀 Starting Memory-Optimized Training")
print("=" * 60)

# Create memory-efficient model
model = create_memory_efficient_model()
model.summary()

# OPTION 1: Try memory-safe progressive unfreezing (might still crash)
try:
    print("\n🔄 Attempting memory-safe progressive unfreezing...")
    history1, history2, final_model = train_memory_safe(model, X_train, y_train, X_val, y_val)
    print("✅ Progressive unfreezing completed successfully!")

except Exception as e:
    print(f"❌ Progressive unfreezing failed: {e}")
    print("\n🔄 Falling back to simple single-phase training...")

    # OPTION 2: Simple single-phase (guaranteed to work)
    model = create_memory_efficient_model()  # Fresh model
    history, final_model = train_simple_effective(model, X_train, y_train, X_val, y_val)
    plot_results(history)

# Final evaluation
print("\n" + "=" * 60)
print("FINAL EVALUATION")
print("=" * 60)

final_metrics = final_model.evaluate(X_val, y_val, verbose=0)
print(f"✅ Final Validation Accuracy: {final_metrics[1]:.4f}")
print(f"✅ Final Validation Precision: {final_metrics[2]:.4f}")
print(f"✅ Final Validation Recall: {final_metrics[3]:.4f}")

if len(final_metrics) > 4:
    print(f"✅ Final Validation AUC: {final_metrics[4]:.4f}")

# Save final model
final_model_path = '/content/drive/MyDrive/accident_detection_models/final_t4_compatible.h5'
final_model.save(final_model_path)
print(f"✅ Final model saved to: {final_model_path}")

Mounted at /content/drive
✅ Dataset loaded successfully!
X_train: (300, 16, 112, 112, 3), y_train: (300,)
X_val: (80, 16, 112, 112, 3), y_val: (80,)
🚀 Starting Memory-Optimized Training
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ DenseNet121 loaded with 428 layers
✅ Initially frozen layers: 428


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 112, 112,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 16, 1024)       │     7,037,504 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 1024)       │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 1024)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16, 128)        │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,696,705 (29.36 MB)

 Trainable params: 656,769 (2.51 MB)

 Non-trainable params: 7,039,936 (26.86 MB)


🔄 Attempting memory-safe progressive unfreezing...
PHASE 1: Training classification head only
Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5271 - loss: 0.7119 - precision: 0.5390 - recall: 0.2861

75/75 ━━━━━━━━━━━━━━━━━━━━ 592s 5s/step - accuracy: 0.5273 - loss: 0.7118 - precision: 0.5395 - recall: 0.2865 - val_accuracy: 0.4750 - val_loss: 0.6959 - val_precision: 0.5000 - val_recall: 0.2381 - learning_rate: 1.0000e-04
Epoch 2/15
50/75 ━━━━━━━━━━━━━━━━━━━━ 1:05 3s/step - accuracy: 0.5456 - loss: 0.6964 - precision: 0.6444 - recall: 0.3295